In [ ]:
from pyrosm import OSM
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, Polygon, MultiPoint
import folium
import matplotlib.pyplot as plt
import alphashape
import contextily 
from descartes import PolygonPatch


In [ ]:
# assign file path to osm pbf file.
base_road_path = 'C:/Users/hular/projects/ClosestDestination/testEnvironment/Data/belfast_slightly_trimmed.osm.pbf'

osm = OSM(base_road_path)

#load road network from graph
G_network = osm.get_network(network_type = 'driving', nodes = True)

# #Convert to ga graph (this will create if not a pyrosm source)
# if not isinstance(G, nx.Graph):
#     G = nx.from_pandas_edgelist(G, source='u', target='v', edge_attr=True)

nodes, edges = osm.get_network(network_type='driving', nodes=True)

#generate graph of network
G = osm.to_graph(nodes, edges, graph_type='networkx')

#edges.plot(figsize=(10,10))

In [ ]:
#load in the data which service areas will be analysed.
#sticking it in a try loop forces it to work.
service_locations_csv = pd.read_csv('C:/Users/hular/projects/ClosestDestination/testEnvironment/Data/libraries_belfast_2024_very_trimmed.csv')

def csv_to_gdf(csv = service_locations_csv, x = 'X COORDINATE', y = 'Y COORDINATE', input_crs = 29902, crs_conversion = None):
    """ function to convert csv to a gdf based off X, Y coordinates and input CRS. Optional CRS conversion."""
    #create a list for each row of geom by zipping and turn into a point tuple
    try:
        csv['geometry'] = list(zip(csv[x], csv[y]))
        csv['geometry'] = csv['geometry'].apply(Point)
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(csv, geometry='geometry', crs=f'EPSG:{input_crs}')
        #Only converts if specified
        if crs_conversion:
            gdf = gdf.to_crs(epsg=crs_conversion)
        return gdf
    except Exception:
        print(f'Exception error: {Exception}')
        
        # Return the GeoDataFrame

service_locations = csv_to_gdf(csv=service_locations_csv, crs_conversion=4326)


In [142]:
#Function to create a dict of names and nearest names to be called in a loop creating the subgraphs for each location.
def nearest_node_and_name(Network_graph, locations, name):
    """ Create a dictionary of location names and nearest node on Graph  """   
    service_xy = {}
    
    for index, row in locations.iterrows():
        #extract x and y for each library
        location_x = row['geometry'].x
        location_y = row['geometry'].y
        #calculate the nearest node on the Graph
        nearest_node = ox.distance.nearest_nodes(Network_graph, location_x, location_y)
        # extract the library name
        name = row['Static Library Name']
        #Combine the name and nearest name. 
        service_xy[name] = {'nearest_node': nearest_node}
    return service_xy

service_locations_nearest_node = nearest_node_and_name(G, service_locations, 'Static Library Name')


{'Ballyhackamore Library': {'nearest_node': 73250694}, 'Finaghy Library': {'nearest_node': 8417220926}, 'Grove Library': {'nearest_node': 3732169380}, 'Woodstock Library': {'nearest_node': 11186018716}}


dict_values([{'nearest_node': 73250694}, {'nearest_node': 8417220926}, {'nearest_node': 3732169380}, {'nearest_node': 11186018716}])

In [ ]:
#Example single not looped yet.
#loop over library nearest node dict with single_source_path_length, 
# create a new dict with name and polygons created from the reachable nodes and retain the travel distance.
#Create closest node for example. Loop this later.
single_location = service_locations.iloc[1]
x,y =(single_location['geometry'].x, single_location['geometry'].y)

nearest_node = ox.nearest_nodes(G, X=x, Y=y )

# distance in metres, time = seconds. For creation of subgraph
max_travel_distance = 1000

# Create a sub-graph and a list of nodes within the specified distance.
#subgraph = subsection of the main Graph based on cutoff
subgraph = nx.single_source_dijkstra_path_length(G, nearest_node, cutoff=max_travel_distance, weight='length')
reachable_nodes = list(subgraph.keys())

node_points = [Point(G.nodes[node]['x'], G.nodes[node]['y']) for node in reachable_nodes]
polygon = Polygon([[point.x, point.y] for point in node_points])


In [ ]:
# Create a gdf for the polygon.
import geopandas as gpd
gdf = gpd.GeoDataFrame([polygon], columns=['geometry'], crs='EPSG:4326')

fig, ax = plt.subplots()
gdf.boundary.plot(ax=ax)
gdf.plot(ax=ax, color='blue', alpha=0.5)
plt.show()



service_point = (54.55998716608919, -5.983139503268679)  # Latitude, Longitude

m = folium.Map(location=[service_point[0], service_point[1]], zoom_start=14)

folium.GeoJson(gdf).add_to(m)

m.save(r"C:\Users\hular\projects\ClosestDestination\testEnvironment\service_area_map.html")

In [ ]:
#convex hull creation. Very ugly. Kind of pointless.
points = MultiPoint(node_points)
convex_hull_polygon = points.convex_hull

#plot convex hull
x,y = convex_hull_polygon.exterior.xy
x_coords = [point.x for point in node_points]
y_coords = [point.y for point in node_points]
plt.figure()
plt.plot(x, y)
plt.scatter(x_coords, y_coords, color='red') # Scatter plot of the original points
plt.show()

In [ ]:
#Convert to pandas series then extract the x and y coords from the list of tuples and then reassign as a list, this removes the 'geometry'.
node_points_series = pd.Series(node_points)
node_point_series_tuples_list = node_points_series.apply(lambda point: (point.x, point.y))
correct_points_list = node_point_series_tuples_list.tolist()

In [ ]:
#alpha param can be defined locally within a region of points. this changes density density off X coordinates density
#lambda ind, r: 500.0 + any(np.array(correct_points_list)[ind][:, 0] == 0.0) #from the docs
alpha_value = 1000

print(alpha_value)
alpha_shape = alphashape.alphashape(correct_points_list, 
                                    alpha_value)

In [ ]:
#create geodataframes alpha hull and points
gdf_alpha = gpd.GeoDataFrame(geometry=[alpha_shape], crs=4326)
points = [Point(xy) for xy in correct_points_list]
gdf_nodes = gpd.GeoDataFrame( geometry= node_points, crs = 4326)

fig, ax = plt.subplots()

# Plot the alpha shape and nodes on same axes
gdf_alpha.plot(ax=ax, color='darkblue', alpha=0.8)  
gdf_nodes.plot(ax=ax, color='red', markersize=5)

ax.set_title(f'Alpha Value = {alpha_value}')
ax.set_xlabel('Long')
ax.set_ylabel('Lat')

plt.show()
